In [53]:
import pandas as pd
import time

In [2]:
import requests
from bs4 import BeautifulSoup

In [5]:
# Add filter condition
page_url = 'https://pubmed.ncbi.nlm.nih.gov/?term=periodontitis&filter=years.1983-1983&page=1'
page = requests.get(page_url)


In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
doc_sum = soup.find_all('a', class_='docsum-title', href=True)

In [8]:
# 한 페이지 내에 있는 제목과 링크를 한 번에 긁어오는 함수 생성
def get_paper_links(page_url):
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    doc_sum = soup.find_all('a', class_='docsum-title', href=True)
    
    titles = []
    links = []
    for doc in doc_sum:
        titles.append(doc.text.strip())
        links.append('https://pubmed.ncbi.nlm.nih.gov'+doc['href'])
    
    return titles, links

In [15]:
page_url = 'https://pubmed.ncbi.nlm.nih.gov/?term=periodontitis&filter=years.1983-1983&page=122'

titles, links = get_paper_links(page_url)

In [18]:
# years list

year_list = [year for year in range(1980, 2022)]
print(year_list)

[1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]


In [50]:
# filter=years.1983-1983

def crawling_year_filter(year):
    titles = []
    links = []
    i = 1
    while True:

        url = f'https://pubmed.ncbi.nlm.nih.gov/?term=periodontitis&filter=years.{year}-{year}&page={i}'

        page_titles, page_links = get_paper_links(url)

        if page_titles == []:
            break

        titles += page_titles
        links += page_links
        
        if i % 10 == 0:
            print(f'PAGE {i} complete')
        i += 1
    
    df = pd.DataFrame(data={'titles':titles, 'links':links, 'year':year})
    df = df.drop_duplicates(subset=['titles'])
    return df

In [51]:
df = pd.DataFrame(columns=['titles', 'links', 'year'])

In [52]:
for year in year_list:
    year_df = crawling_year_filter(year)
    df = pd.concat([df, year_df], axis=0)
    
    print(f"{year} Complete")
    
    time.sleep(30)

PAGE 10 complete
PAGE 20 complete
PAGE 30 complete
PAGE 40 complete
PAGE 50 complete
PAGE 60 complete
PAGE 70 complete
PAGE 80 complete
PAGE 90 complete
PAGE 100 complete
1980 Complete


NameError: name 'time' is not defined

In [55]:
print(len(df))
df.head()

1030


,titles,links,year
0,Juvenile periodontitis.,https://pubmed.ncbi.nlm.nih.gov/6988466/,1980
1,[Chronic apical periodontitis].,https://pubmed.ncbi.nlm.nih.gov/7008097/,1980
3,"Juvenile periodontitis. Some microbiological, ...",https://pubmed.ncbi.nlm.nih.gov/6928856/,1980
4,Tooth mobility and periodontal therapy.,https://pubmed.ncbi.nlm.nih.gov/6938529/,1980
5,Periodontal probing: what does it mean?,https://pubmed.ncbi.nlm.nih.gov/7000852/,1980


In [56]:
def crawling_keywords(url):
    # url Html 구조 긁어오기
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        div_abstract = soup.find('div', class_='abstract')   
        div_abstract.find('div', class_='abstract-content selected').decompose()

        keywords = div_abstract.find('p').get_text(strip=True)
        keywords = keywords.split(':')[1]
        meta_keywords = keywords = keywords.split('; ')
        meta_keywords = ','.join(meta_keywords)
    except:
        meta_keywords = 'Error'

    return meta_keywords


In [60]:
df['meta_keywords'] = ''

In [61]:

for i, url in enumerate(df.links):
    keywords = crawling_keywords(url)
    df['meta_keywords'][i] = keywords
    
    print(f'PAGE {i}/{len(df)} complete')
    

PAGE 0/1030 complete
PAGE 1/1030 complete
PAGE 2/1030 complete
PAGE 3/1030 complete
PAGE 4/1030 complete


KeyboardInterrupt: 

In [62]:
df

,titles,links,year,meta_keywords
0,Juvenile periodontitis.,https://pubmed.ncbi.nlm.nih.gov/6988466/,1980,Error
1,[Chronic apical periodontitis].,https://pubmed.ncbi.nlm.nih.gov/7008097/,1980,Error
3,"Juvenile periodontitis. Some microbiological, ...",https://pubmed.ncbi.nlm.nih.gov/6928856/,1980,Error
4,Tooth mobility and periodontal therapy.,https://pubmed.ncbi.nlm.nih.gov/6938529/,1980,Error
5,Periodontal probing: what does it mean?,https://pubmed.ncbi.nlm.nih.gov/7000852/,1980,
...,...,...,...,...
1035,[Principles of the TCP-implantation system aft...,https://pubmed.ncbi.nlm.nih.gov/7004847/,1980,
1036,[Use of floctafenine for dental pain of adults...,https://pubmed.ncbi.nlm.nih.gov/6109234/,1980,
1037,Dynamic histomorphometry of alveolar bone remo...,https://pubmed.ncbi.nlm.nih.gov/7416512/,1980,
1038,Systemic distribution of N2 paste containing 1...,https://pubmed.ncbi.nlm.nih.gov/6935586/,1980,


## 실행 x

In [ ]:
df = pd.DataFrame(data={'titles':titles, 'links':links, 'meta_keywords':meta_keywords})

df.iloc[:10]

# 파일이 저장되어 있지 않은 관계로 예전 파일 사용
target_df = pd.read_csv("../files/test_df.csv")

target_df.head()

# target_df : A
# df : B
pd.merge(target_df, df, left_on='title', right_on='titles', how='right')